# 数学优化建模简介

## 目标和先决条件

该建模示例的目的是在混合整数规划（MIP）问题的公式化中介绍关键元素。对于MIP问题公式的每个组成部分，我们提供描述，相关的Python代码以及描述该组成部分的数学符号。

为了完全理解此部分的内容，读者应：


* 熟悉Python。
* 有相关学科背景，包括但不限于工程、计算机科学、经济学、统计学等各类“硬核”科学，或包含定量模型和方法的任一学科。

读者还应查询 Gurobi Python API 的相关 [文档](https://www.gurobi.com/resources/?category-filter=documentation)。此外，在《混合整数线性编程的系列教程》视频中详细介绍了此文档的内容。

你可以在 [这里](https://www.gurobi.com/resource/tutorial-mixed-integer-linear-programming/) 观看视频教程

**注意：** 你可以通过单击 [此处](https://github.com/arvinxx/gurobi-and-mathematical-modeling/archive/master.zip) 下载包含此示例和其他示例的代码仓库。为了正确运行此 Jupyter Notebook，您必须具有 Gurobi 许可证。如果您没有，则可以 *商业用户身份* 申请[评估许可证](https://www.gurobi.com/downloads/request-an-evaluation-license/)，或以 *学术用户* 的身份下载 [免费许可证](https://www.gurobi.com/academia/academic-program-and-licenses)。

## 问题描述

假如有一家咨询公司，它有三个空缺职位：Tester（测试）、Java Developer（Java开发）和 Architect（架构师）。这三个职位的候选人是：Carlos、Joe 和 Monika。
这家咨询公司对每个候选人进行了能力测试，以评估他们完成每项职位的能力。这些测试的结果称为 「匹配分数」（matching scores）。假设一个职位只能分配一个候选人，并且最多可以为一个候选人分配一个职位。

我们的目标是：确定候选人和职位的分配，以使每个职位都得到满足。每个候选人最多分配一个岗位，分配后的总「匹配得分」最大。


## 数学优化

数学优化（也称为数学编程）是一种声明性方法，其中建模者制定了一个优化问题，该问题捕获了复杂决策问题的关键特征。然后 Gurobi 使用最先进的数学和计算机科学技术来解决这一类数学优化问题。

数学优化模型具有五个组成部分：

* 数据集（Sets）
* 参数(Parameters)
* 决策变量(Decision variables)
* 约束（Constraints）
* 目标函数（Objective function(s)）


以下 Python 代码导入了 Gurobi 模块，并将 `GRB` 类导入了主命名空间。


In [78]:
import gurobipy as gp
from gurobipy import GRB

## 候选人分配问题

### Data

列表 $R$ 包含三个候选人的名称：Carlos，Joe 和 Monika。
列表 $J$ 包含职位的名称：Tester（测试），Java Developer（Java开发） 和 Architect（架构）。

$r \in R$：索引和候选人集。候选人 $r$ 属于候选人集 $R$。

$j \in j$：索引和职位集。职位 $j$ 属于职位集 $J$。






In [79]:
# 候选人集和职位集
R = ['Carlos', 'Joe', 'Monika']
J = ['Tester', 'JavaDeveloper', 'Architect']

以下匹配得分表中列出了每种候选人参与每种职位的能力：

![scores](https://gw.alipayobjects.com/zos/antfincdn/NWZW0Xv%24Ms/1d4ac105-34ee-446b-8974-77cdc9410763.png)

对于每个候选人 $r$ 和职位 $j$，都有一个对应的匹配得分 $s$。匹配的分数 $s$ 只能接受 0 到 100 之间的值。

也就是说，对于 $r \in R $ 和 $j \in J$ ，都有 $s_{r,j} \in [0,100]$ 。

我们使用 Gurobi Python 的`multidict` 函数来初始化一个或多个字典。该函数以字典为参数。键代表候选人和职位的可能组合。

In [80]:
# 匹配分数数据
combinations, scores = gp.multidict({
    ('Carlos', 'Tester'): 53,
    ('Carlos', 'JavaDeveloper'): 27,
    ('Carlos', 'Architect'): 13,
    ('Joe', 'Tester'): 80,
    ('Joe', 'JavaDeveloper'): 47,
    ('Joe', 'Architect'): 67,
    ('Monika', 'Tester'): 53,
    ('Monika', 'JavaDeveloper'): 73,
    ('Monika', 'Architect'): 47
})

以下构造函数创建一个空的 Model 对象 `m` 。我们通过传递字符串 `"RAP"` 作为参数来指定模型名称。 Model 对象 `m` 包含一个优化问题。该优化问题由一组决策变量，一组约束和目标函数组成。

In [81]:
# 声明并初始化模型
m = gp.Model('RAP')

## 决策变量

为了解决此分配问题，我们需要确定将哪个候选人分配给哪个职位。我们为职位的每种可能候选人分配引入一个决策变量。因此，我们有 9 个决策变量。

为了简化模型公式的数学符号，我们定义了以下候选人和职位的指标:

![variables](https://gw.alipayobjects.com/zos/antfincdn/LEzDic7PJB/81d8f5f2-7a92-49c4-b3a0-11879715e2fd.png)

例如，$x_{2,1}$ 是与将候选人 Joe 分配给职位岗位 Tester 的决策变量。因此，如果$r \in R$ 被分配给 $j \in J$，则决策变量 $x_{r,j}$ 等于1，否则为0。

`Model.addVars()` 方法为 Model 对象创建决策变量。此方法返回包含新创建的决策变量。该变量类型为 Gurobi 专有的 `tupledict` 对象。我们将 `combinations` 对象作为指定变量索引的第一个参数。`name` 关键字用于为新创建的决策变量命名。默认情况下，变量被假定为非负数。


In [82]:
# 为RAP模型创建决策变量
x = m.addVars(combinations, name="assign")

## 职位限制

现在我们讨论与职位相关的约束。这些约束条件需要确保每个职位都恰好由一个候选人安排。

Tester 职位的约束要求将候选人1（Carlos），候选人2（Joe）或候选人3（Monika）分配给该 职位。这对应于以下约束。

约束 (Tester=1)

$$
x_{1,1} + x_{2,1} + x_{3,1} = 1
$$

类似地，Java职位和架构师职位的约束可以定义如下。

约束 (Java Developer = 2)

$$
x_{1,2} + x_{2,2} + x_{3,2} = 1
$$

约束 (Architect = 3)

$$
x_{1,3} + x_{2,3} + x_{3,3} = 1
$$

职位约束由下表的列定义。

![jobs](https://gw.alipayobjects.com/zos/antfincdn/wCDVAxhuEU/642de4a4-eae6-496b-8237-98a3acebf283.png)

通常情况下，可以将 Tester 职位的约束定义如下。

$$
x_{1,1} + x_{2,1} + x_{3,1} = \sum_{r=1}^{3 } x_{r,1} =  \sum_{r \in R} x_{r,1} = 1
$$
可以以类似简洁的方式定义所有职位约束。对于$j \in J$，取所有候选人的决策变量的总和。我们可以如下编写相应的职位约束。

$$
\sum_{r \in R} x_{r,j} = 1
$$

Gurobi 的 `Model.addConstrs()` 方法定义了 Model 对象 `m` 的职位约束。此方法返回职位约束的变量，该变量类型也为 `tupledict`。


In [83]:
# 创建职位约束
jobs = m.addConstrs((x.sum('*',j) == 1 for j in J), name='job')

此方法的第一个参数 `x.sum(‘*’, j)` 是 sum 方法，它定义职位约束的 LHS 如下：

对于职位 $J$ 中的每个职位 $j$ ，取所有候选人上决策变量的总和。`==` 定义了一个相等约束，数字 1 是这些约束的RHS。

这些约束意味着每个职位只能分配一个候选人。第二个参数是此类约束的名称。

## 候选人约束


候选人约束需要确保至多为每个候选人分配一个职位。也就是说，可能不是所有的候选人都被分配了。
比如这样约束：Carlos 最多分配一个职位: 职位1(Tester)、职位2 (Java)或职位3(架构师)。这个约束如下书写：

约束 (Carlos=1)

$$
x_{1, 1} + x_{1, 2} + x_{1, 3}  \leq 1.
$$

这个约束使它们的值小于或等于1，以允许 Carlos 不被分配到任何职位的可能性。同样，对候选人 Joe 和 Monika 的约束可以定义如下:

约束 (Joe=2)

$$
x_{2, 1} + x_{2, 2} + x_{2, 3}  \leq 1.
$$

约束 (Monika=3)

$$
x_{3, 1} + x_{3, 2} + x_{3, 3}  \leq 1.
$$

请注意，候选人约束是由下表的行定义的。

![resources](https://gw.alipayobjects.com/zos/antfincdn/Kaf%26AdwmYU/0dd89ba0-5688-4618-b3dd-4c26571e976f.png)

候选人 Carlos 的约束可以定义如下。

$$
x_{1, 1} + x_{1, 2} + x_{1, 3} = \sum_{j=1}^{3 } x_{1,j} = \sum_{j \in J} x_{1,j} \leq 1.
$$

同样，每个约束都可以用简洁的方式书写。对于$r \in R$，对所有职位的决策变量求和。我们可以将相应的候选人约束写如下。

$$
\sum_{j \in J} x_{r,j} \leq  1.
$$

Gurbo 的 `Model.addConstrs()` 方法定义了 Model 对象 `m` 的候选人约束。
此方法的第一个参数 `x.sum(r, ‘*’)` 是求和方法，它定义候选人约束的LHS如下： 对于候选人集 $R$ 中的每个候选人$r$，对所有职位进行决策变量的求和。`<=`定义了一个更少或相等的约束，约束的 RHS 是 1。这些限制意味着每个候选人最多只能分配1个职位。

第二个参数是此类约束的名称。


In [84]:
# 创建候选人约束
resources = m.addConstrs((x.sum(r,'*') <= 1 for r in R), name='resource')

## 目标函数

目标函数是使满足职位和候选人约束的职位的总匹配得分最大化。

对于测试人员职位，如果分配了候选人Carlos，其匹配分数为 $53x_{1,1}$, 如果分配了候选人 Joe，匹配分数为 $80x_{2,1}$, 如果分配了候选人 Monika，匹配分数为  $53x_{3,1}$。

因此，Tester职位的匹配分数如下，其中该总和中的决策变量只有一项值为 1。

$$
53x_{1,1} + 80x_{2,1} + 53x_{3,1}. 
$$

类似地，Java开发人员和架构师职位的匹配分数定义如下。Java开发人员职位的匹配分数是:

$$
27x_{1, 2} + 47x_{2, 2} + 73x_{3, 2}.
$$

架构师职位的匹配分数是:

$$
13x_{1, 3} + 67x_{2, 3} + 47x_{3, 3}.
$$

总匹配得分为下表中每个单元格的总和。

![objfcn](https://gw.alipayobjects.com/zos/antfincdn/irHnhJepQ8/188b942e-a9d6-4248-8cfb-54accd1c84fb.png)

目标是使分配的总匹配分数最大化。因此，目标函数定义如下。

\begin{equation}
\text{Maximize} \quad (53x_{1,1} + 80x_{2,1} + 53x_{3,1}) \; +
\end{equation}

\begin{equation}
\quad (27x_{1, 2} + 47x_{2, 2} + 73x_{3, 2}) \; +
\end{equation}

\begin{equation}
\quad (13x_{1, 3} + 67x_{2, 3} + 47x_{3, 3}).
\end{equation}

目标函数中括号中的每一项可以表示为:

\begin{equation}
(53x_{1,1} + 80x_{2,1} + 53x_{3,1}) = \sum_{r \in R} s_{r,1}x_{r,1}.
\end{equation}

\begin{equation}
(27x_{1, 2} + 47x_{2, 2} + 73x_{3, 2}) = \sum_{r \in R} s_{r,2}x_{r,2}.
\end{equation}

\begin{equation}
(13x_{1, 3} + 67x_{2, 3} + 47x_{3, 3}) = \sum_{r \in R} s_{r,3}x_{r,3}.
\end{equation}

因此，目标函数可以简写为：

\begin{equation}
\text{Maximize} \quad \sum_{j \in J} \sum_{r \in R} s_{r,j}x_{r,j}.
\end{equation}

Gurobi 的 `Model.setobobjective()` 方法定义了 Model 对象 `m` 的目标函数。

In [85]:
# 目标:使所有职位的总匹配分数最大化
m.setObjective(x.prod(scores), GRB.MAXIMIZE)

第一个参数指定了目标函数的表达式。

> 注意：匹配的分数参数 score 和分配决策变量 x 都是在 `combinations` 的键上定义的。因此，我们可以使用`x.prod(score)` 的方式,得到 score 矩阵与 x 变量矩阵的元素乘法的和。

第二个参数`GRB.MAXIMIZE`， `MAXIMIZE`，是优化核心目标。在这种情况下，我们想要 **最大化** 所有分配的总匹配分数。

我们使用 Gurobi 的 `write()` 方法将模型公式写入 `RAP.lp` 文件。

In [86]:
# 保存模型以供查看
m.write('RAP.lp')

![RAP](https://gw.alipayobjects.com/zos/antfincdn/J6j1Hm9Kis/8afcfa53-9d62-462d-a228-de904d9c72ed.png)

我们使用 Gurobi 的 `optimize()` 方法来为模型对象 `m` 定义的问题求解。

In [87]:
# 运行优化引擎
m.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 6 rows, 9 columns and 18 nonzeros
Model fingerprint: 0xb6602fb2
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 6 rows, 9 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.6000000e+32   1.800000e+31   4.600000e+02      0s
       5    1.9300000e+02   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds
Optimal objective  1.930000000e+02


使用 Gurobi 的`Model.getVars()` 方法来检索模型对象 `m` 中所有变量。

`.x` 属性用于查询当前可行解下的变量值，`.varName` 属性用于获得决策变量的名称。

In [88]:
# 显示决策变量的值
for v in m.getVars():
    if v.x > 1e-6:
        print(v.varName, v.x)

# 显示最佳总匹配分数
print('匹配得分总分: ', m.objVal)

assign[Carlos,Tester] 1.0
assign[Joe,Architect] 1.0
assign[Monika,JavaDeveloper] 1.0
匹配得分总分:  193.0


最佳分配方案是：

* Carlos -> Tester, 匹配得分为 53
* Joe -> Architect, 匹配得分为 of 67
* Monika ->  Java Developer, 匹配得分为 73.

最大匹配得分的总分为193分。


## 具有预算约束的候选人分配问题

现在，假设存在一个固定成本 $C_{r,j}$，将 $r \in R$ 分配给$j \in J$。另外一个条件则是：可以用于职位分配的预算 $B$ 是有限的。

将Carlos、Joe 或 Monika 分配给任何职位的成本分别为 $\$1000$、$\$2000$ 和 $\$3000$。可用预算为$\$5000$。

### 数据


列表 $R$ 包含三个候选人的名称: Carlos、Joe 和 Monika。
列表 $J$ 包含职位职位的名称：Tester，Java Developer 和 Architect。

我们使用Gurobi的 `multidict` 方法初始化两个字典：
* `scores` 定义每种候选人和职位组合的匹配分数。
* `cost` 定义为职位分配候选人相关的固定成本。


In [89]:
# 候选人和职位集
R = ['Carlos', 'Joe', 'Monika']
J = ['Tester', 'JavaDeveloper', 'Architect']

# 匹配分数数据
# cost 以千美元为单位
combinations, scores, costs = gp.multidict({
    ('Carlos', 'Tester'): [53, 1],
    ('Carlos', 'JavaDeveloper'): [27, 1],
    ('Carlos', 'Architect'): [13,1],
    ('Joe', 'Tester'): [80, 2],
    ('Joe', 'JavaDeveloper'): [47, 2],
    ('Joe', 'Architect'): [67, 2],
    ('Monika', 'Tester'): [53, 3] ,
    ('Monika', 'JavaDeveloper'): [73, 3],
    ('Monika', 'Architect'): [47, 3]
})

# 可用预算（千美元）
budget = 5

以下构造函数创建一个空的 Model 对象 `m` 。我们通过传递字符串 `"RAP2"` 作为参数来指定模型名称。 Model 对象 `m` 包含一个优化问题。该优化问题由一组决策变量，一组约束和目标函数组成。

In [90]:
# 声明和初始化模型
m = gp.Model('RAP2')

### 决策变量

如果 $r \in R$ 被分配给 $j \in J$，则决策变量 $x_{r,j}$ 为1，否则为0。

`Model.addVars()` 方法定义模型对象“ m”的决策变量。

由于存在预算限制，因此可能不会安排所有职位。为了解决这个问题，我们定义了一个新的决策变量，该变量指示是否安排了职位。

如果职位 $j \in J$ 没有被安排，$g_{j}$ = 1，否则为0。这个变量是一个中间变量，它指示一个职位不被安排。

***备注：*** 对于 `RAP` 的先前公式，我们将赋值变量定义为非负且连续的，这是`Model.addVars()` 方法的 `vtype` 参数的默认值。

然而，在 RAP2 中，由于我们添加到模型中的预算约束，我们需要显式地将这些变量定义为布尔变量。在 `addvars()` 中添加 `vtype=GRB.BINARY` 参数，将该决策变量定义为布尔类型。

In [91]:
# 为 RAP2 模型创建决策变量
x = m.addVars(combinations, vtype=GRB.BINARY, name="assign")

# 为 RAP2 模型创建差距变量
g = m.addVars(J, name="gap")

### 职位限制

由于我们预算有限，无法为职位分配候选人，因此可能无法安排所有职位。对于职位约束，存在两种可能性，即该职位有候选人安排，或者没有被安排。因此我们需要声明一个变量 $g_j$ 捕获后一种可能性。因此，职位约束如下。

对于 $j \in J$，必须给职位分配一个候选人，或者必须将相应的 $g_j$ 变量设为1:

$$
\sum_{r \: \in \: R} x_{r,\; j} + g_{j} = 1.
$$


In [92]:
# 创建职位限制
jobs = m.addConstrs((x.sum('*',j) + g[j]  == 1 for j in J), name='job')

### 候选人限制

候选人约束需要确保至多为每个候选人分配一个职位。也就是说，可能不是所有的候选人都被分配了。因此，将候选人约束写如下。

对于$r \in R$，最多只能给该候选人分配一个职位:

$$
\sum_{j \: \in \: J} x_{r,\; j} \leq 1.
$$

In [1]:
# 创建候选人约束
resources = m.addConstrs((x.sum(r,'*') <= 1 for r in R), name='resource')

NameError: name 'm' is not defined

### 预算约束

该约束条件确保分配候选人以满足职位要求的成本不超过可用预算。任务和预算的成本以千美元计。

如果分配了候选人 Carlos，则安排 Tester 职位的成本为 $1x_{1,1}$，如果分配了候选人 Joe，则成本为 $2x_{2,1}$，如果为候选人 Monika，则为 $3x_{3,1}$。

因此，完成Tester职位的成本如下，其中该总和中的至多一项不为零。

$$
1x_{1,1} + 2x_{2,1} + 3x_{3,1}. 
$$

同样，安排 Java Developer 和 Architect 职位的成本定义如下。

安排 Java Developer 职位的成本是：

$$
1x_{1, 2} + 2x_{2, 2} + 3x_{3, 2}.
$$

安排 Architect 职位的成本为：

$$
1x_{1, 3} + 2x_{2, 3} + 3x_{3, 3}.
$$

因此，填补职位空缺的总成本应该小于或等于可用的预算。

\begin{equation}
(1x_{1,1} + 2x_{2,1} + 3x_{3,1}) \; +
\end{equation}

\begin{equation}
(1x_{1, 2} + 2x_{2, 2} + 3x_{3, 2}) \; +
\end{equation}

\begin{equation}
(1x_{1, 3} + 2x_{2, 3} + 3x_{3, 3}) \leq 5
\end{equation}

预算约束中的括号中的每一项可以表示为:

\begin{equation}
(1x_{1,1} + 2x_{2,1} + 3x_{3,1}) = \sum_{r \in R} C_{r,1}x_{r,1}.
\end{equation}

\begin{equation}
(1x_{1, 2} + 2x_{2, 2} + 3x_{3, 2}) = \sum_{r \in R} C_{r,2}x_{r,2}.
\end{equation}

\begin{equation}
(1x_{1, 3} + 2x_{2, 3} + 3x_{3, 3}) = \sum_{r \in R} C_{r,3}x_{r,3}.
\end{equation}

因此，预算约束可以简写为:

\begin{equation}
\sum_{j \in J} \sum_{r \in R} C_{r,j}x_{r,j} \leq B.
\end{equation}

代码如下：

In [3]:
budget = m.addConstr((x.prod(costs) <= budget), name='budget')

NameError: name 'm' is not defined


Gurobi 的 `Model. addconstr()` 方法定义了 Model 对象 `m` 的预算约束。

此方法的第一个参数 `x.prod(costs)` 定义了预算约束的LHS。 `<=` 定义了一个更少或相等的约束，并且 `budget` 是约束的RHS。
该约束条件是分配候选人以满足职位需求的总成本不能超过可用预算。

第二个参数是此约束的名称。

## 目标函数

目标函数类似于 RAP。目标中的第一项是职位的总匹配分数。在这个 RAP 的扩展中，可能不是所有的职位都被填满了;然而，我们要严厉惩罚这种可能性。为了达到这个目的，我们在目标函数中有第二项，它将所有职位岗位的缺口变量的总和乘以一个比较大的惩罚值 $M$。

目标功能类似于 RAP。目标中的第一项是职位的总匹配分数。在 RAP2 中，可能并非所有职位都已完成。因此，我们要严重惩罚这种可能性。为此目的，我们在目标函数中有第二项，它求和所有职位上的差距变量的总和，然后乘以一个大的罚分$M$。

注意，匹配分数的最大值是100，而我们给$M$的值是101。$M$ 值背后的基本原理是，巨大的差值大大降低了总匹配分数值。

因此，目标函数是最大化职位的总匹配分数减去差距变量等于1的惩罚值。
$$
\max \; \sum_{j \; \in \; J} \sum_{r \; \in \; R} s_{r,j}x_{r,j} -M \sum_{j \in J} g_{j}
$$

In [95]:
# 因未提供职位空缺而受到的惩罚
M = 101

In [96]:
# 目标：最大化职位的总匹配分数
# 空缺职位将受到惩罚
m.setObjective(x.prod(scores) - M*g.sum(), GRB.MAXIMIZE)

In [97]:
# 运行优化引擎
m.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 7 rows, 12 columns and 30 nonzeros
Model fingerprint: 0xf3c6f8c8
Variable types: 3 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Presolve time: 0.00s
Presolved: 7 rows, 12 columns, 30 nonzeros
Variable types: 0 continuous, 12 integer (12 binary)
Found heuristic solution: objective 52.0000000

Root relaxation: objective 1.350000e+02, 4 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  135.00000    0    2   52.00000  135.00000   160%     -    0s
     0     0  121.66667    0    7   52.00000  121.66667   134%     -    0s
     0     0     cutoff    0      

目标函数的定义包括不安排职位的惩罚。然而，我们感兴趣的是当所有的职位不被填满时的最优总匹配分值。为此，我们需要使用匹配分值 $s_{r,j}$ 和分配决策变量 $x_{r,j}$ 计算总匹配分值。

In [98]:
# 根据分配变量计算总匹配得分
total_matching_score = 0
for r, j in combinations:
    if x[r, j].x > 1e-6:
        print(x[r, j].varName, x[r, j].x) 
        total_matching_score += scores[r, j]*x[r, j].x

print('Total matching score: ', total_matching_score)  

assign[Joe,Tester] 1.0
assign[Monika,JavaDeveloper] 1.0
Total matching score:  153.0


### 分析

回想一下，预算是$\$5,000$，与分配三个候选人相关的总成本是$\$6,000$。这意味着没有足够的预算来分配我们拥有的三种候选人。因此，Gurobi 优化器必须选择两个候选人来满足职位需求（放着一个职位没人干），并使总匹配分数最大化。

最大匹配的两个分数分别是80%( Joe-> 测试人员)和73% (Monika -> Java开发人员)。最低的分数是13%(建筑师职位的Carlos)。

将 Joe 分配给测试人员的职位，将Monika分配给Java开发人员的职位，而没有人分配给架构师的职位需要花费 $\$5,000$，并且产生的匹配总分为153。这是由 Gurobi 优化器找到的最优解决方案。

Copyright © 2020 Gurobi Optimization, LLC

翻译 By Arvin Xu